In [2]:
import pandas as pd
import binascii

In [3]:
fname="/Users/jianlin/Documents/Project_STDF_wafermap/STDF_data/main_Lot_1_Wafer_1_Oct_13_09h33m41s_STDF.stdf"

In [718]:
with open(fname,mode='rb') as file:
    row_data = file.read()
str_row_data=str(row_data)
str_row_data

"b'\\x02\\x00\\x00\\n\\x02\\x04y\\x00\\x00\\x14\\xf8T\\xffWtdataformatter tester config; Ver: SmarTest s/w rev. 7.1.4.3 (E), 15-Oct-12; dataformatter 2.0.9 Oct 15 2012 18:37:15\\x82\\x00\\x01\\n\\xeaT\\xffW\\xf8T\\xffW\\x01 N \\xff\\xff \\x011\\x00\\x15localhost.localdomain\\t93000-SOC\\x05cp_32\\x00\\x00\\x07vincent\\x0593000 s/w rev. 7.1.4.3 (E), 15-Oct-12\\n\\x00\\x0525.25\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x14\\x00\\x01P\\x01\\x02\\x01\\x01\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x12\\x00\\x01<\\x01\\x00\\x01\\x00\\x03Xin\\x0510101\\x011\\x01\\x01\\x13\\x00\\x01<\\x02\\x00\\x02\\x00\\x04Xout\\x0510102\\x012\\x01\\x01\\x0e\\x00\\x01<\\x03\\x00\\x00\\x00\\x03VCC\\x0211\\x00\\x01\\x01\\x14\\x00\\x02\\x1e\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00 \\x00\\x80\\x00\\x80  \\x07\\x00\\x02\\n\\x01\\x02\\xf8T\\xffW\\x00\\x02\\x00\\x05\\n\\x01\\x01\\x12\\x00\\x01<\\x01\\x00\\

# Dict for convert

In [155]:
dict_code_ctype = {
    "U1": "B",
    "U2": "H",
    "U4": "I",
    "U8": "Q",
    "I1": "b",
    "I2": "h",
    "I4": "i",
    "I8": "q",
    "R4": "f",
    "R8": "d",
    "B1": "B",
    "C1": "c",
    "N1": "B"
}
dict_code_ctype['U1']

'B'

In [ ]:
dict_header_grep = {
    'FAR':'x02\x00\x00\n',
    'ATR':''
}

# Filter Method

In [261]:
dict_pack = {
    'FAR':[0,10],
    'ATR':[0,20],
    'MIR':[1,10],
    'MRR':[1,20],
    'PCR':[1,30],
    'HBR':[1,40],
    'SBR':[1,50],
    'PMR':[1,60],
    'PGR':[1,62],
    'PLR':[1,63],
    'RDR':[1,70],
    'SDR':[1,80],
    'WIR':[2,10],
    'WRR':[2,20],
    'WCR':[2,30],
    'PIR':[5,10],
    'PRR':[5,20],
    'TSR':[10,30],
    'PTR':[15,10],
    'MPR':[15,15],
    'FTR':[15,20],
    'BPS':[20,10],
    'EPS':[20,20],
    'GDR':[50,10],
    'DTR':[50,30]
}
dict_pack['PRR'][0]

5

In [744]:
def struct_packing(rtype,subtype):
    packing = str(struct.pack("BB",rtype,subtype))
    return packing[2:len(packing)-1]

def data_mark(string_row_data,key_header,key):
    key_list = string_row_data.split(key_header)    #will cut key_header
    
    for num in range(0, len(key_list)-1):
        group_count = 0
        group_count = group_count + len(key_list[num])
        
        #out of range
        if debug_by_positionrule(key,group_count)==0:
            pass
        #in rule
        else:
            key_list[num] = key_list[num][:len(key_list[num])-8] + "|" + key_list[num][len(key_list[num])-8:]
    for num in range(0, len(key_list)-1):
        group_count_2 = 0
        group_count_2 = group_count_2 + len(key_list[num])
        
        #out of range
        if debug_by_positionrule(key,group_count_2)==0:
            key_list[num] = key_list[num]+key_header
        #in rule
        else:
            key_list[num] = key_list[num]+"%"+key+":"+key_header
    
        #key_list[num] = key_list[num]+":"+key
    
    return "%".join(key_list)

def debug_by_positionrule(key_judge,group_count):
    #position locate
    dict_position = {}
    for key in dict_pack.keys():
        dict_position[key]= processed_data.find(struct_packing(dict_pack[key][0],dict_pack[key][1]))
    #ATR:bewteen FAR,MIR
    if group_count > dict_position[key_judge]:
        return 0 #fail
    else:
        return 1 #pass


In [736]:
def data_mark(string_row_data,key_header,key):
    key_list = string_row_data.split(key_header)
    for num in range(0, len(key_list)-1):
        key_list[num] = key_list[num][:len(key_list[0])-8] + "|" + key_list[0][len(key_list[0])-8:]
    for num in range(0, len(key_list)-1):
        key_list[num] = key_list[num]+":"+key
    return ":".join(key_list)

In [635]:
struct_packing(0,20)

'\\x00\\x14'

In [739]:
debug_by_positionrule('ATR',100)

42

In [743]:
# read binary data
with open(fname,mode='rb') as file:
    row_data = file.read()
str_row_data=str(row_data)

# data mark
for num_key in range(0,25):
    key = list(dict_pack.keys())[num_key]
    header = struct_packing(dict_pack[key][0],dict_pack[key][1])
    str_row_data = data_mark(str_row_data,header,key)
processed_data = str_row_data
processed_data

10
36
374
186
692
2450
2951
92
2743
47
629
312
270
1629
47
533
53
55
195
53
61
440
809
2665
722
847
317
125
118
118
118
118
118
118
118
119
119
119
1069
123
130
123
123
123
123
123
123
123
124
124
124
343
467
726


"b'|\\x02\\x00%FAR:\\x00\\n%\\x02\\|x04y\\x00%ATR:\\x00\\x14%\\xf8T\\xffWtdataformatter tester config; Ver: SmarTest s/w rev. 7.1.4.3 (E), 15-Oct-12; dataformatter 2.0.9 Oct 15 2012 18:37:15|\\x82\\x00\\x01\\n%\\xeaT\\xffW\\xf8T\\xffW\\x01 N \\xff\\xff \\x011\\x00\\x15localhost.localdomain\\t93000-SOC\\x05cp_32\\x00\\x00\\x07vincent\\x0593000 s/w rev. 7.1.4.3 (E), 15-Oct-12\\n%\\x00\\x0525.25\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x14%\\x00\\x01P%\\x01\\x02\\x01\\x01\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x12\\x00\\x01<%\\x01\\x00\\x01\\x00\\x03Xin\\x0510101\\x011\\x01\\x01|\\x13\\x00%PMR:\\x01<%\\x02\\x00\\x02\\x00\\x04Xout\\x0510102\\x012%\\x01\\x01|\\x0e\\x00%PMR:\\x01<%\\x03\\x00\\x00\\x00\\x03VCC\\x0211\\x00\\x01\\x01\\x14%\\x00\\x02\\x1e%%\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00 \\x00\\x80\\x00\\x80  \\x07\\x00\\x02\\n%%\\x01\\x02\\xf8T\\xffW\\x00\\x02\\x00\\x05\

15-Oct-12\\n\\x00\\x0525.25\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00:ATR

In [690]:
def extract_data(processed_data):
    record_type_split = processed_data.split("|")
    for data in range(1,len(record_type_split)):
        #print(record_type_split[data])
        data_extracted = record_type_split[data].split("%")
        print(len(data_extracted))
        print(data_extracted)
    return data_extracted

In [616]:
extracted = extract_data(processed_data)
extracted

3
['\\x02\\x00', 'FAR', '\\x02\\']
3
['x04y\\x00', 'ATR', '\\xf8T\\xffWtdataformatter tester config; Ver: SmarTest s/w rev. 7.1.4.3 (E), 15-Oct-12; dataformatter 2.0.9 Oct 15 2012 18:37:15']
3
['\\x82\\x00', 'MIR', '\\xeaT\\xffW\\xf8T\\xffW\\x01 N \\xff\\xff \\x011\\x00\\x15localhost.localdomain\\t93000-SOC\\x05cp_32\\x00\\x00\\x07vincent\\x0593000 s/w rev. 7.1.4.3 (E), ']
3
['15-Oct-1', 'GDR', '\\x00\\x0525.25\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00']
2
['\\x00\\x00', '']
4
['ATR', '\\x00', 'SDR', '\\x01\\x02\\x01\\x01\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00']
3
['\\x12\\x00', 'PMR', '\\x01\\x00\\x01\\x00\\x03Xin\\x0510101\\x011\\x01\\x01']
3
['\\x13\\x00', 'PMR', '\\x02\\x00\\x02\\x00\\x04Xout\\']
3
['x0510102', 'SBR', '\\x01\\x01']
3
['\\x0e\\x00', 'PMR', '\\x03\\x00\\x00\\x00\\x03VCC\\x0211']
2
['\\x00\\x01', '']
4
['MRR', '\\x00', 'WCR', '\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00 \\x

['\\x07\\x00', 'MRR', "1U\\xffW \\x00\\x00'"]

In [615]:
extracted[15]

IndexError: list index out of range

In [434]:
df = pd.DataFrame(columns=['rec_length','rec_type','rec_body'])
df.append(a)

,0,rec_body,rec_length,rec_type
0,\x1a\x00,NaN,NaN,NaN
1,PRR,NaN,NaN,NaN
2,\x01\x01\x00\x00\x00\x01\x00\x01\x00\xfb\xff\x...,NaN,NaN,NaN


In [593]:
a = [1,2,3]
b = [4,5,6]

In [596]:
rec_length = []
for line in extracted:
    print(line)

TypeError: 'NoneType' object is not iterable